In [ ]:
### Autoreload all modules ###
%load_ext autoreload
%autoreload 2


### Import libraries ###
import os
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
#from skimage import filters
from pathlib import Path


### Set base path ###
base_path = Path(os.getcwd())
while not (base_path / '.git').exists():
    base_path = base_path.parent
print('Base path: ', base_path)


### Import custom modules ###
sys.path.append(str(base_path / 'src/data/datasets/'))
sys.path.append(str(base_path / 'src/training/'))
sys.path.append(str(base_path / 'src/evaluation/'))
sys.path.append(str(base_path / 'src/visualization/'))
sys.path.append(str(base_path / 'src/models/'))

from dataset import UkraineDataset
from baselines import spatial_correction, nbr
from pixel_metrics import ConfuseMatrixMeter
from model_eval_plots import percentile_stretch

In [ ]:
root_path = str(base_path / "data/UKR/final_datasets/change_new/")
ukr_dataset = UkraineDataset(root_path, mode = "test", normalize = False,  return_cloud_mask = True, 
                 sentinel_type = "S2", indices = None, dilate_mask = True, bands = [1, 2, 3, 4, 5, 6, 7], file_type = "tif")


## 1. CVA

In [39]:
# loop over the dataset
cm = ConfuseMatrixMeter(n_class=2)
PLOT = False

for i in range(len(ukr_dataset)):

    imageA = ukr_dataset[i]['A'].numpy()
    imageB = ukr_dataset[i]['B'].numpy()
    mask = ukr_dataset[i]['mask'].numpy()
    clouds_A = ukr_dataset[i]['cloud_mask_A'].numpy()
    clouds_B = ukr_dataset[i]['cloud_mask_B'].numpy()
    combined_cloud_mask = (clouds_A == 0) & (clouds_B == 0)

    # calculate the euclidean distance
    eu_distance = np.linalg.norm(imageA - imageB, axis = 0) 
    eu_distance = np.where(combined_cloud_mask, eu_distance, np.nan)

    # apply thresholding
    threshold = filters.threshold_otsu(eu_distance[~np.isnan(eu_distance)])
    distance_binary = eu_distance > threshold

    # apply spatial correction
    distance_binary = spatial_correction(distance_binary, radius=3)

    # pixel confusion matrix
    temp_cm = confusion_matrix(mask.flatten(), distance_binary.flatten(), labels = [0, 1])
    cm.update(temp_cm)

    # perform percentile stretch on RG
    rgb_A = np.stack([percentile_stretch(imageA[0], 1, 99), percentile_stretch(imageA[1], 1, 99), percentile_stretch(imageA[2], 1, 99)], axis = 0)
    rgb_B = np.stack([percentile_stretch(imageB[0], 1, 99), percentile_stretch(imageB[1], 1, 99), percentile_stretch(imageB[2], 1, 99)], axis = 0)


    if PLOT:
        fig, ax = plt.subplots(1, 5, figsize=(20, 5))
        ax[0].imshow(rgb_A.transpose(1, 2, 0))
        ax[1].imshow(rgb_B.transpose(1, 2, 0))
        ax[2].imshow(mask)
        ax[3].imshow(eu_distance)  
        ax[4].imshow(distance_binary)
        plt.show()

In [40]:
cm.get_scores()

{'acc': 0.804257033465432,
 'miou': 0.4047117683375748,
 'mf1': 0.4510365563011507,
 'iou_0': 0.8040498057418598,
 'iou_1': 0.005373730933289694,
 'F1_0': 0.8913830982291331,
 'F1_1': 0.01069001437316832,
 'precision_0': 0.9990015792800022,
 'precision_1': 0.005395739693488482,
 'recall_0': 0.8046964455617224,
 'recall_1': 0.5684896405283996}

## 2. NBR

In [3]:
cm = ConfuseMatrixMeter(n_class=2)
PLOT = False

for i in range(len(ukr_dataset)):

    imageA = ukr_dataset[i]['A'].numpy()
    imageB = ukr_dataset[i]['B'].numpy()
    mask = ukr_dataset[i]['mask'].numpy()
    clouds_A = ukr_dataset[i]['cloud_mask_A'].numpy()
    clouds_B = ukr_dataset[i]['cloud_mask_B'].numpy()
    combined_cloud_mask = (clouds_A == 0) & (clouds_B == 0)

    # calculate the NBR
    nbr_A = nbr(imageA)
    nbr_B = nbr(imageB)

    # calculate absolute difference
    distance = nbr_A - nbr_B 
    distance = np.where(combined_cloud_mask, distance, np.nan)

    # apply thresholding
    distance_binary = distance > 0.1

    # apply spatial correction
    distance_binary = spatial_correction(distance_binary, radius=1)

    # pixel confusion matrix
    temp_cm = confusion_matrix(mask.flatten(), distance_binary.flatten(), labels = [0, 1])
    cm.update(temp_cm)

    rgb_A = np.stack([percentile_stretch(imageA[0], 1, 99), percentile_stretch(imageA[1], 1, 99), percentile_stretch(imageA[2], 1, 99)], axis = 0)
    rgb_B = np.stack([percentile_stretch(imageB[0], 1, 99), percentile_stretch(imageB[1], 1, 99), percentile_stretch(imageB[2], 1, 99)], axis = 0)

    if PLOT:
        fig, ax = plt.subplots(1, 6, figsize=(20, 5))
        ax[0].imshow(rgb_A.transpose(1, 2, 0))
        ax[1].imshow(rgb_B.transpose(1, 2, 0))
        ax[2].imshow(nbr_A, cmap = "RdYlGn")
        ax[3].imshow(nbr_B, cmap = "RdYlGn")
        ax[4].imshow(mask)
        ax[5].imshow(distance_binary)
        plt.show()


In [4]:
cm.get_scores()

{'acc': 0.9094604758902373,
 'miou': 0.4605855649948816,
 'mf1': 0.4879358270048154,
 'iou_0': 0.909362412190966,
 'iou_1': 0.011808717798797223,
 'F1_0': 0.9525298598163014,
 'F1_1': 0.023341794193329424,
 'precision_0': 0.9991438731955551,
 'precision_1': 0.011909896178207958,
 'recall_0': 0.9100715368629497,
 'recall_1': 0.5815945530402584}